In [ ]:
# TODO: make sure the ps3 is is in my global memeory.
# TODO:  make sure room_id and baord_id are in the global params
# TODO: Make sure the viewport is also passed and in the global memory
# TODO: Ask Luc why this works when creating a stickie (state = {'col': 'red', 'font': '45', 'tex': 'yo'})
# TODO: Tell Luc about no owner when stickies are created programmatically



In [8]:
from pydantic_ai import Agent, RunContext


In [1]:
import json
from uuid import UUID
import nest_asyncio
nest_asyncio.apply()

In [2]:
room_id, board_id = "e07aa346-37dd-4bf5-90bf-fc59a1c3b2e9/d12ec467-6f82-49e0-873a-e15df43e52c4".split("/")
room_id, board_id

('e07aa346-37dd-4bf5-90bf-fc59a1c3b2e9',
 'd12ec467-6f82-49e0-873a-e15df43e52c4')

In [3]:
cd ../


/Users/mahdi/Documents/GitHub/next/foresight


In [4]:

import uuid

from foresight.config import config as conf, prod_type

from foresight.Sage3Sugar.pysage3 import PySage3
ps3 = PySage3(conf, prod_type)

Configuring ps3 client ... 
Completed configuring Sage3 Client


In [ ]:
# ps3.s3_comm.get_users()

In [5]:
b = ps3.rooms[room_id].boards[board_id]
sb = list(b.smartbits)
sb


[(UUID('6050dcb0-3d75-4688-9bb7-5ba7e5b65c3d'),
  Stickie(path='', touched=set(), app_id=UUID('6050dcb0-3d75-4688-9bb7-5ba7e5b65c3d'), data=Data(path='.data', touched=set(), position=Position(path='.data.position', touched=set(), x=1501610.3777184172, y=1500330.5077368016, z=0.0), size=Size(path='.data.size', touched=set(), width=200.0, height=200.0, depth=0.0), rotation=Rotation(path='.data.rotation', touched=set(), x=0.0, y=0.0, z=0.0), raised=True, type='Stickie'), tags=[], state=StickieState(path='.state', touched=set(), text='Sushi', color='yellow', fontSize=36, lock=False))),
 (UUID('e80db8dd-305c-4e84-80ad-27e82ba98706'),
  Webview(path='', touched=set(), app_id=UUID('e80db8dd-305c-4e84-80ad-27e82ba98706'), data=Data(path='.data', touched=set(), position=Position(path='.data.position', touched=set(), x=1501088.1685693709, y=1500671.5616962556, z=0.0), size=Size(path='.data.size', touched=set(), width=435.0, height=420.0, depth=0.0), rotation=Rotation(path='.data.rotation', touch

In [ ]:
# sb = b.smartbits[UUID('58c42c91-2a70-426d-a43f-0285901358b7')]
# sb.state.text = "Hi There"
# sb.send_updates()

In [ ]:
ps3.get_smartbits_by_type("stickie", room_id, board_id)

In [ ]:
### Creating Stuff

state = {"text": "Python said Yolo", 'color': 'orange', 'fontSize': '45'}
data = {'size': {'width': 400, 'height': 200, 'depth': 0}, 'position': {'x':1501140.0, "y":1500150.0+100, 'z':0.0}}
response = ps3.create_app(room_id, board_id, 'Stickie', state, data)


In [ ]:
ps3.get_smartbits_by_type("stickie", room_id, board_id)

In [ ]:
state = {"webviewurl": "https://sage-3.github.io/"}
data = {'size': {'width': 500, 'height': 500, 'depth': 0}, 'position': {'x':1501618.0491539645, 'y':1500355.010865956, 'z':0.0}}
res = ps3.create_app(room_id, board_id, 'Webview', state, data)

In [ ]:
res

In [12]:
from pydantic import BaseModel, Field
from typing import Literal, List, Union
from enum import Enum

from typing import List
from pydantic import BaseModel, HttpUrl, Field
from typing import Dict
import time
from typing import Dict, List


from enum import Enum

In [ ]:
class StickieColor(str, Enum):
    """
    Enumeration of predefined stickie colors.
    """
    RED = "red"
    GREEN = "green"
    BLUE = "blue"
    YELLOW = "yellow"

class StickieState(BaseModel):
    """
    Model representing the state of a stickie note that encapsulates the content and appearance details of a stickie note.
    """
    text: str = Field(..., description="The text that stick contains")
    color: StickieColor = Field(default=StickieColor.YELLOW, description="The color of the stickie to create")
    fontsize: str = Field(..., description="str representation of text size. Use 45 if not provided")


class Position(BaseModel):
    """
    Model representing the position of a Smartbit application on the board.
    """

    x: float = Field(..., description="The x position of the app")
    y: float = Field(..., description="The y position of the app")
    z: float = Field(default=0.0, description="The z position of the app")


class Size(BaseModel):
    """
    Model representing the dimensions of a Smartbit application.
    """

    width: float = Field(default=200, description="The width of the app")
    height: float = Field(default=200, description="The height of the app")
    depth: float = Field(default=0, description="The depth of the app")        
        
class Data(BaseModel):
    """
    Model encapsulating additional Smartbit application data including size and position.

    Attributes:
        size (Size): The dimensions of the application.
        position (Position): The position of the application on the board.
    """
    size: Size
    position: Position    
        

class SmarbitType(str, Enum):
    """
    Enumeration of available Smartbit application types in SAGE3.
    """
    STICKIE = "Stickie"
    # WEBVIEW = "Webview" 
        
class SmarbitApp(BaseModel):
    """
    Model defining a Smartbit application.
    This model represents a software Smartbit application that resides on a specialized desktop interface - Sage3 wall.

    Attributes:
        smartbit_type (SmarbitType): The type of smartbit application.
        state (Union[StickieState]): The state of the application. Typically, a StickieState.
        data (Data): Additional data for the application, including its size and position.
    """

    smartbit_type: SmarbitType
    state: Union[StickieState]
    data: Data



class AppId(BaseModel):
    """
    Model representing the unique identifier for a Smartbit application.
    """
    app_id: UUID
    

# class WebViewState(BaseModel):
#     """The data that is required to create a webview, this is what is used ot open
#     webview, browser or  a webpage"""
#     webviewurl: HttpUrl = Field(default="https://google.com")

# class WallAIMessage(BaseModel):
#     "The message for the AI regarding a query"
#     message: str

        
# class WallAIMessage(BaseModel):
#     reponse: Union[WallAIMessage, SmarbitApp]                

In [7]:
import os
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your Anthropic API key: ")
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your openAI API key: ")



Enter your openAI API key:  ········


In [ ]:
# sage_smarbits_description = """You are an AI assistant that open application (we sometime also refer to them as "smartbits" or apps) for the SAGE3 operating system. Users interact with SAGE3 through a display wall or board; this is similar to a desktop for an OS."""

# prompt_create_app = f"""{sage_smarbits_description}
# Your job is to listen to users' natural language requests and translate them into the precise JSON parameters needed to 
# create (may refer to it as opne) or remove(may refer to it as delete or close) apps  on the SAGE3 wall. 
# When users describe what they want to create, you'll convert their request into the proper technical format that SAGE3 requires to call the app. 
# You understand all SAGE3 smartbit capabilities and will recommend the best app type for each user's needs.

# To find a suitable location to create an app, please use the get_available_space function.
# To create an app, use create_apps function, passing it the list of apps that need ot be created. 


# """

In [ ]:
prompt_create_app = """You are an AI assistant for the SAGE3 operating system, responsible for opening (or creating) and closing (or removing) integrated applications (also called "smartbits") in the Sage3 operating system.
Users interact with SAGE3 through a display wall or board; this is similar to a desktop for an OS. 
Listen to users' natural language requests and translate them into precise JSON parameters for opening (or creating) or closing (or removing) apps.
When a user describes an app to be created, convert their request into the technical format required by SAGE3 and recommend the best app type based on the user's needs. 

Use the get_available_space function to determine a suitable location for the app.
Use the create_apps function to create the app by passing a list of apps.
"""

create_delete_app_agent = Agent(  
    "claude-3-5-haiku-latest",
    result_type=SmarbitApp,    
    system_prompt=prompt_create_app,
     model_settings={'temperature': 0.0}    
)
@create_delete_app_agent.tool_plain 
def get_available_space(nb_apps: int) -> Position:
    """
    Return positions for placing Smartbit apps on the board.

    Generates a list of Position objects to be used in a Smartbit app's data.

    Args:
        nb_apps (int): Number of positions required.

    Returns:
        List[Position]: A list of position objects.
    """
    
    return [Position(**{'x': 1501973.615403275, 'y': 1500349.2037941106, 'z': 0})] * nb_apps


@create_delete_app_agent.tool_plain 
def create_apps(sb_list: List[SmarbitApp]) -> str:
    """
    Create Smartbit apps from the provided initialization data.

    Iterates over each SmarbitApp object, creates an app via ps3.create_app, and returns a message 
    indicating success or failure for each creation.

    Args:
        sb_list (List[SmarbitApp]): List of SmartbitApp objects.

    Returns:
        List[str]: Messages for each app creation attempt.
    """
    returned_messages = []
    for sb in sb_list:
        sb_type = sb.smartbit_type
        state = json.loads(sb.state.model_dump_json())
        data = json.loads(sb.data.model_dump_json())
    
        response = ps3.create_app(room_id, board_id, sb_type, state, data)
        if response and type(response.json()) is dict and response.json().get('success', None):
            returned_messages.append(f"App of type {sb_type} with  state:{state} created successfully")
        else:
            # TODO:fix in ps3.create_app 
            returned_messages.append(f"Couldn not create app of type {sb_type} with state:{state}. Error: {response.json()['error']}")
    return returned_messages



NameError: name 'Agent' is not defined

In [ ]:
result = await create_delete_app_agent.run("create Two stickies, with popular pizza sauce names")

In [ ]:
users= ps3.s3_comm.get_users()
users

In [ ]:
# sage_smarbits_description =  """The SAGE3 operating system lets users interact with applications (called "smartbits") through a display wall or board."""

# prompt_select_apps = f"""
# {sage_smarbits_description} As an AI assistant, your role is to help users find the exact application(s) they need by intepreting their natural language requests and returning only the app_ids that precisely match their requirements.

# When users describe what they're looking for, you need to ensure each suggested application id retured meets their specific criteria.

# You can use any of the aviable tools to interact with the wall, include to list currently available apps.

# Some selection operations may require multiple steps, so you need to think step by step. 

# If none of the apps matches the exact requirements specified in the user query, then return an empty list.
# """

In [3]:
prompt_select_apps = """You are an AI assistant for the SAGE3 operating system. 
The SAGE3 operating system lets users interact with applications (called "smartbits") through a display wall or board. 
Your task is to interpret natural language requests and return only the app_ids of applications that exactly meet the specified criteria. 

Use all available tools, including those for listing currently available apps.
Some complex selection operations may require multiple steps. So you need break down the operations into step by step process.
If no app precisely matches the request, return an empty list.
"""


select_app_agent = Agent(  
    "openai:gpt-4o",
    result_type=List[AppId],    
    system_prompt=prompt_select_apps,
     model_settings={'temperature': 0.6}    
)

@select_app_agent.tool_plain
def get_apps_for_context() -> Dict:
    """
    Get sanitized details for Smartbit applications on the SAGE3 wall.

    Fetches raw app data from PySage3, removes internal fields (e.g., _id, _updatedBy, rotation),
    formats selected fields (e.g., creation timestamp, creator name), and omits apps with unrecognized types.

    Returns:
        Dict: A mapping of app IDs to their sanitized details.
    """
    
    users= ps3.s3_comm.get_users()
    apps = ps3.get_apps(room_id, board_id)
    new_fromatted_apps = {}
    for k, v in apps.items():
        try:
            SmarbitType(v['data']['type'])
            del v['_id']
            del v['_updatedBy']
            del v['_updatedAt']
            del v['data']["rotation"]
            del v['data']["raised"]
            del v['data']["pinned"]
            del v['data']["dragging"]

            if "sources" in v['data']['state']:
                del v['data']['state']["sources"]

            if "lock" in v['data']['state']:
                del v['data']['state']["lock"]
            
            if "executeInfo"  in v['data']['state']:
                del v['data']['state']["executeInfo"]        
        
            user = users.get(v['_createdBy'], ("Unknown", ))
            v['_createdBy'] = user[0]        
            v['_createdAt'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(v['_createdAt']// 1000))                
            new_fromatted_apps[k] = v
        except Exception as e:
            print(f"Ommitted {v['data']['type']}")
        
    return new_fromatted_apps




NameError: name 'Agent' is not defined

In [ ]:
get_apps_for_context()

In [ ]:
results = await select_app_agent.run("What stickies are relevant to Moroccan food")
results.data

In [ ]:
b.smartbits[UUID('2a8cbd8e-9905-435f-ac78-8aa0d876093d')]

In [ ]:
results = await select_app_agent.run("Which apps were created by Mahdi")
results.data

In [ ]:
type(ps3.get_apps(room_id, board_id))

In [ ]:
get_app_details([UUID('00592a68-2700-4b69-abb4-340f631fa295'), UUID('e907cca5-e928-441f-8dc6-1343d27e3a44')])

In [ ]:
AppId(app_id = UUID('00592a68-2700-4b69-abb4-340f631fa295'))  == AppId(app_id = UUID('00592a68-2700-4b69-abb4-340f631fa295'))

In [ ]:
#results  = await select_app_agent.run("What stickies are PURPLE")
results.data

In [ ]:
a = []
type(a) is list

In [ ]:
# # Assuming sage_smarbits_description is defined somewhere
# destructive_operations_prompt = f"""
# {sage_smarbits_description}.  

# When deleting or moving SAGE3 wall applications (smartbits), first display their app_ids and identifying details. Only proceed after receiving explicit user confirmation. If confirmation is not given, cancel the operation and notify the user that no changes will occur. Use available wall interaction tools to retrieve application details as needed.
# If user cancels, be VERY brief and concise and do NOT offer to help the user with something else after.
# Use select_apps_to_delete to rreturn the list of AppId to remve
# You can use get_app_details to get the details of an app, which you need to show the user for confirmation

# Use delete_apps to delete the app once use has confirmed  deletion

# """

In [17]:
destructive_operations_prompt = """You are an AI assistant for the SAGE3 operating system. 
The SAGE3 operating system lets users interact with applications (called "smartbits") through a display wall or board. 
When the user requests to delete or move applications (smartbits) using natural language requests, use the following 4 sequential steps to ensure a smooth process:
step 1: Use the select_apps_to_delete tool to return the list of app_ids carefully selected for removal and get_app_details tool get their details.
step 2: Display the selected app_ids and their details to the user and ask for confirmation.
step 3: Wait for explicit user approval before proceeding with any deletion or movement.
step 4: If the user confirms, proceed with the operation using the delete_apps tool. Otherwise, notify the user briefly that no changes will occur.
"""

destructive_operations_agent = Agent(  
    "claude-3-5-haiku-latest",
    result_type=str,    
    system_prompt=destructive_operations_prompt,
    model_settings={'temperature': 0.5}    
)

@destructive_operations_agent.tool_plain
async def select_apps_to_delete(query_filter: str) -> List[AppId]:
    """
    Select Smartbit apps for deletion using the provided filter.

    Queries the SAGE3 wall for apps matching the filter and returns their AppId objects.
    Returns None if the result format is unexpected.

    Parameters:
        query_filter (str): Filter condition for selecting apps.

    Returns:
        List[AppId] or None: IDs of the selected apps, or None on error.
    """
    print(f"query_filter is {query_filter}")
    result = await select_app_agent.run(query_filter)
    if result and type(result.data) is not list:
        print(f"Error executing query to select apps to delete {result.data}")
        return None
    elif len(result.data) > 0 and type(result.data[0]) is not AppId:
        print(f"Error returning the select app as an AppId")
        return None
    return result.data
        
@destructive_operations_agent.tool_plain
def get_app_details(app_ids: List[AppId]) -> Dict:
    """
    Retrieve details for the specified Smartbit apps.

    Returns a dictionary mapping app IDs (as strings) to their details.

    Parameters:
        app_ids (List[AppId]): List of app IDs.

    Returns:
        Dict: Mapping of app IDs to their details.
    """
    print(f"get_app_details called with {app_ids}")
    all__apps = ps3.get_apps(room_id, board_id)
    return   {k:v for k,v in ps3.get_apps(room_id, board_id).items() if AppId(app_id = UUID(k)) in app_ids}
    
@destructive_operations_agent.tool_plain
def delete_apps(app_ids_to_remove: List[AppId]) -> bool:
    """
    Delete the specified Smartbit apps.

    Parameters:
        app_ids_to_remove (List[AppId]): IDs of apps to delete.

    Returns:
        bool: True if all apps were deleted successfully, False otherwise.
    """
    print(f"removing apps {app_ids_to_remove}")
    return True
   
    


In [ ]:
# %%capture ouput
# result = await destructive_operations_agent.run("can you delete apps with ids [UUID('00592a68-2700-4b69-abb4-340f631fa295'), UUID('e907cca5-e928-441f-8dc6-1343d27e3a44')]")
# print(result.data)

In [24]:
result = await destructive_operations_agent.run("delete all red Stickies")
result.data

query_filter is type:Sticky color:red


'I found 2 red Stickies:\n1. A Sticky with text "Marinara Sauce"\n2. A Sticky with text "Japan"\n\nWould you like me to proceed with deleting these red Stickies? Please confirm (yes/no).'

In [23]:
%%capture output
result_2 = await destructive_operations_agent.run("Yes, please delete them", message_history=result.new_messages())
result_2.data

In [22]:
output.stdout

"removing apps [AppId(app_id=UUID('2a8cbd8e-9905-435f-ac78-8aa0d876093d')), AppId(app_id=UUID('61e9c037-6c0b-48fe-b47c-9f18140de7b3'))]\n"